# Explore the simple-sr parameters save in HDF5 format

In [1]:
import numpy as np
import h5py
import pprint
import json

## From keras/saving/legacy/hdf5_format.py:
### load_weights_from_hdf5_group

In [2]:
f = h5py.File('simple-sr.h5')
list(f.attrs), list(f.keys())

(['backend', 'keras_version', 'model_config', 'training_config'],
 ['model_weights', 'optimizer_weights'])

In [3]:
f.attrs['backend'], f.attrs['keras_version']

('tensorflow', '2.12.0')

## Get the model config

In [4]:
model_config = json.loads(f.attrs['model_config'])
pprint.pp(model_config)

{'class_name': 'Sequential',
 'config': {'name': 'sequential',
            'layers': [{'class_name': 'InputLayer',
                        'config': {'batch_input_shape': [None, 124, 129, 1],
                                   'dtype': 'float32',
                                   'sparse': False,
                                   'ragged': False,
                                   'name': 'input_1'}},
                       {'class_name': 'Resizing',
                        'config': {'name': 'resizing',
                                   'trainable': True,
                                   'dtype': 'float32',
                                   'height': 32,
                                   'width': 32,
                                   'interpolation': 'bilinear',
                                   'crop_to_aspect_ratio': False}},
                       {'class_name': 'Normalization',
                        'config': {'name': 'normalization',
                                   

## Get the training config

In [5]:
training_config = json.loads(f.attrs['training_config'])
pprint.pp(training_config)

{'loss': {'class_name': 'SparseCategoricalCrossentropy',
          'config': {'reduction': 'auto',
                     'name': 'sparse_categorical_crossentropy',
                     'from_logits': True,
                     'ignore_class': None}},
 'metrics': [[{'class_name': 'MeanMetricWrapper',
               'config': {'name': 'accuracy',
                          'dtype': 'float32',
                          'fn': 'sparse_categorical_accuracy'}}]],
 'weighted_metrics': None,
 'loss_weights': None,
 'optimizer_config': {'class_name': 'Custom>Adam',
                      'config': {'name': 'Adam',
                                 'weight_decay': None,
                                 'clipnorm': None,
                                 'global_clipnorm': None,
                                 'clipvalue': None,
                                 'use_ema': False,
                                 'ema_momentum': 0.99,
                                 'ema_overwrite_frequency': None,
   

## Get the layer names

In [6]:
def load_attributes_from_hdf5_group(group, name):
    """Loads attributes of the specified name from the HDF5 group.

    This method deals with an inherent problem
    of HDF5 file which is not able to store
    data larger than HDF5_OBJECT_HEADER_LIMIT bytes.

    Args:
        group: A pointer to a HDF5 group.
        name: A name of the attributes to load.

    Returns:
        data: Attributes data.
    """
    if name in group.attrs:
        data = [
            n.decode("utf8") if hasattr(n, "decode") else n
            for n in group.attrs[name]
        ]
    else:
        data = []
        chunk_id = 0
        while "%s%d" % (name, chunk_id) in group.attrs:
            data.extend(
                [
                    n.decode("utf8") if hasattr(n, "decode") else n
                    for n in group.attrs["%s%d" % (name, chunk_id)]
                ]
            )
            chunk_id += 1
    return data

<HDF5 group "/model_weights" (11 members)>

In [22]:
model_weights = f['model_weights']
layer_names = load_attributes_from_hdf5_group(model_weights, 'layer_names')
layer_names

['resizing',
 'normalization',
 'conv2d',
 'conv2d_1',
 'max_pooling2d',
 'dropout',
 'flatten',
 'dense',
 'dropout_1',
 'dense_1']

## Get the weights

In [23]:
def load_subset_weights_from_hdf5_group(f):
    """Load layer weights of a model from hdf5.

    Args:
        f: A pointer to a HDF5 group.

    Returns:
        List of NumPy arrays of the weight values.

    Raises:
        ValueError: in case of mismatch between provided model
            and weights file.
    """
    weight_names = load_attributes_from_hdf5_group(f, "weight_names")
    return [np.asarray(f[weight_name]) for weight_name in weight_names]

In [24]:
for layer_name in layer_names:
    group = model_weights[layer_name]
    data = load_subset_weights_from_hdf5_group(group)
    print('%s:' % layer_name)
    print('data sets: %d' % len(data))
    for dset in data:
        print('======> %s' % str(dset.shape))
        print('%s' % dset)
    print('')

resizing:
data sets: 0

normalization:
data sets: 3
======> (1,)
[0.12540944]
======> (1,)
[0.58403146]
======> ()
102374400

conv2d:
data sets: 2
======> (3, 3, 1, 32)
[[[[-0.23518433  0.08655639  0.13882366 -0.07313813  0.07969236
    -0.26233104  0.10715329 -0.23484035  0.20917654 -0.1767307
    -0.23540823  0.0402841   0.08439577  0.095152    0.2046656
     0.19635804  0.0481615   0.21123488 -0.04234381 -0.02780385
     0.1419905   0.0603185   0.06638119  0.23312676  0.04398374
     0.10858666  0.13165264 -0.18748036  0.15512642 -0.14523187
    -0.05994598  0.2050475 ]]

  [[ 0.09234667  0.1869532  -0.0179678  -0.15386316  0.18082677
    -0.21368691  0.03889729  0.02836575  0.0807576   0.1350901
    -0.09493233 -0.15250754  0.12201102  0.11525422  0.04330232
     0.08376915 -0.28328654 -0.10311655 -0.06221682 -0.15759712
     0.09805828  0.16834138  0.17557923 -0.12731767  0.08426783
     0.15764311  0.13855942  0.16104302 -0.11222859 -0.09622579
     0.05867683  0.04588864]]

  [[